**Трейдер**.

Для успешного выполнения этого задания вам предлагается “залезть в шкуру” трейдера. Имеются исторические данные по курсам валют (подсказка: ищите нужные данные в Example 2)  от ЦБ России. Пусть ваша команда выберет 5 валют (выберите любые валюты, которые вам интересны) и найдите ответы на следующие вопросы:

* для каждых последовательных 7 дней (например: 01.01.2000-07.01.2000, 02.01.2000-08.01.2000 и т. п.) найдите самую выгодную валюту - ту, относительно которой все остальные падали.

* для каждого 30-дневного промежутка определите, на сколько процентов вы могли бы увеличить свой капитал за 30 дней, если бы знали курсы валют наперёд - считайте, что вечером вы можете перекладывать все ваши средства в одну или несколько валют, найдите самый выгодный такой период.

* найдите наиболее и наименее коррелированные (по отношению друг к другу) валюты в каждом календарном году. Для этого посчитайте коэффициенты корреляции по данным за все 365/366 дней каждого года. Попробуйте выдвинуть гипотезы, почему случались те или иные взлёты/падения, в зависимости от политических и исторических событий.

In [ ]:
!pip install pandas_read_xml

     |████████████████████████████████| 63.8MB 58kB/s 
     |████████████████████████████████| 512kB 43.5MB/s 
     |████████████████████████████████| 133kB 44.9MB/s 
ERROR: kaggle 1.5.8 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.10 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
  Found existing installation: pytz 2018.9
    Uninstalling pytz-2018.9:
      Successfully uninstalled pytz-2018.9
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import pandas as pd
import pandas_read_xml as pdx


# common code

currency_codes = {
    "R01215": "Датская крона",
    "R01310": "Исландская крона", 
    "R01035": "Фунт стерлингов Соединенного королевства", 
    "R01010": "Австралийский доллар", 
    "R01235": "Доллар США",
}


def get_url(from_date: str, to_date: str, code: str) -> str:
  """Create Url with date range and country code."""
  return f"http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={from_date}&date_req2={to_date}&VAL_NM_RQ={code}"


def create_and_clean_df(url: str) -> pd.DataFrame:
  """Create Dataframe from XML and convert `Value` to float."""
  df = pdx.read_xml(url, ['ValCurs', 'Record'])
  df['Value'] = [x.replace(',', '.') for x in df['Value']]
  df['Value'] = df['Value'].astype(float)
  return df


1. для каждых последовательных 7 дней (например: 01.01.2000-07.01.2000, 02.01.2000-08.01.2000 и т. п.) найдите самую выгодную валюту - ту, относительно которой все остальные падали.

In [ ]:
from typing import Dict


def get_name_profitable(
    from_date: str,
    to_date: str,
    currency_codes: Dict[str, str]
) -> str:
  """Calculate the most profitable currency for a given period."""
  max_delta = float("-inf")
  name_profitable = ''
  for currency_code, currency_name in currency_codes.items():
    data = create_and_clean_df(get_url(from_date, to_date, currency_code))
    data['Value'] = data['Value'].astype(float)
    data['Nominal'] = data['Nominal'].astype(float)
    data['Value'] =  data['Value'] / data['Nominal']
    count_days = len(data['Value'])
    delta = data['Value'][count_days-1] / data['Value'][0]
    if max_delta < delta:
      max_delta = delta
      name_profitable = currency_name
  return name_profitable

def print_profitable_result(from_date, to_date, currency_codes):
  print(f"""Самая выгодная валюта за период с {from_date} до {to_date}
{get_name_profitable(from_date, to_date, currency_codes)}""")


In [ ]:
from_date = "01/03/2001"
to_date = "07/03/2001"

print_profitable_result(from_date, to_date, currency_codes)

Самая выгодная валюта за период с 01/03/2001 до 07/03/2001
Фунт стерлингов Соединенного королевства


In [ ]:
from_date = "01/03/2002"
to_date = "07/03/2002"

print_profitable_result(from_date, to_date, currency_codes)

Самая выгодная валюта за период с 01/03/2002 до 07/03/2002
Фунт стерлингов Соединенного королевства


In [ ]:
from_date = "01/03/2005"
to_date = "07/03/2005"

print_profitable_result(from_date, to_date, currency_codes)

Самая выгодная валюта за период с 01/03/2005 до 07/03/2005
Исландская крона


2. для каждого 30-дневного промежутка определите, на сколько процентов вы могли бы увеличить свой капитал за 30 дней, если бы знали курсы валют наперёд - считайте, что вечером вы можете перекладывать все ваши средства в одну или несколько валют, найдите самый выгодный такой период.

In [ ]:
dates = [
  ("02/03/2001", "02/04/2001"),
  ("02/04/2001", "02/05/2001"),
  ("02/05/2001", "02/06/2001"),
]

In [ ]:
from pandas._typing import FrameOrSeries
from typing import List, Dict


def buy_currency(money: float, percent: float) -> float:
  """Get currency profit for tommorow."""
  money *= (1+percent)
  return money


def get_value_pct_change(df: pd.DataFrame) -> FrameOrSeries:
  """Get percentage of change from one day to another."""
  return df['Value'].pct_change()


def calculate_max_profit_period(
    dates: List[tuple], 
    currency_codes: Dict[str, str]
) -> str:
  """Calculate period with maximum possible profit from currency trading."""
  max_profit = 0
  max_profit_period = None
  
  for date_tuple in dates:
    money = 100.0
    changes = []
    for currency in currency_codes:
      url = get_url(*date_tuple, code=currency)
      df = create_and_clean_df(url)
      changes.append(get_value_pct_change(df))

    data = pd.DataFrame([df.values for df in changes])

    for day in data:
      max_value = data[day].max()
      if max_value and max_value > 0:
        money = buy_currency(money, max_value)

    if money > max_profit:
      max_profit = money
      max_profit_period = date_tuple

  answer = f"""Период с максимальной вожможной прибылью: 
{max_profit_period[0]} - {max_profit_period[1]} - {max_profit-100}%"""
  return answer



In [ ]:
print(calculate_max_profit_period(dates, currency_codes))

Период с максимальной вожможной прибылью: 
02/04/2001 - 02/05/2001 - 15.554178548688284%


3. найдите наиболее и наименее коррелированные (по отношению друг к другу) валюты в каждом календарном году. Для этого посчитайте коэффициенты корреляции по данным за все 365/366 дней каждого года. Попробуйте выдвинуть гипотезы, почему случались те или иные взлёты/падения, в зависимости от политических и исторических событий.


In [ ]:
def get_currency_names(correlated: pd.DataFrame, position:int = 0) -> str:
  """Get currency name from correlated key."""
  correlation_key = correlated.keys()[0][position]
  return currency_codes[correlation_key]


def calculate_currency_correspondence(
    currency_codes: Dict[str, str], 
    date_start: str, 
    date_to: str
) -> str:
  """Calculate the most and the least corresponding currencies."""
  dfs = []
  for code in currency_codes:
    url = get_url(date_start, date_to, code)
    df = create_and_clean_df(url)['Value']
    dfs.append(df)
    
  data = pd.DataFrame([df.values for df in dfs], index=currency_codes)

  correlation = data.T.corr().unstack()
  correlation = correlation[correlation<1]

  sorted_correlation = correlation.sort_values(ascending=True)

  least_correlated = sorted_correlation[:1]
  most_correlated = sorted_correlation[-1:]

  least_corr_first_name = get_currency_names(least_correlated)
  least_corr_second_name = get_currency_names(least_correlated, position=1)

  most_corr_first_name = get_currency_names(most_correlated)
  most_corr_second_name = get_currency_names(most_correlated, position=1)

  answer = f"""Наименее зависимые валюты за период {date_start} - {date_to} - {least_corr_first_name} / {least_corr_second_name} со значением {least_correlated[0]}
Наиболее зависимые валюты за период {date_start} - {date_to} - {most_corr_first_name} / {most_corr_second_name} со значением {most_correlated[0]}
"""
  return answer

In [ ]:
date_start = "02/03/2000"
date_to = "02/03/2001"
print(calculate_currency_correspondence(currency_codes, date_start, date_to))


Наименее зависимые валюты за период 02/03/2000 - 02/03/2001 - Доллар США / Австралийский доллар со значением 0.47461263769007994
Наиболее зависимые валюты за период 02/03/2000 - 02/03/2001 - Австралийский доллар / Исландская крона со значением 0.9387706732989517



1)
2)На курс исландской кроны наибольшее влияние оказало создание социал-демокртаического альянса в 2000 году, который объединил в себе 4 партии. Из экономических факторов влияние оказали: рост уровней инфляции и безработицы.

In [ ]:
date_start = "02/03/2001"
date_to = "02/03/2002"
print(calculate_currency_correspondence(currency_codes, date_start, date_to))

Наименее зависимые валюты за период 02/03/2001 - 02/03/2002 - Австралийский доллар / Исландская крона со значением -0.29824471254077545
Наиболее зависимые валюты за период 02/03/2001 - 02/03/2002 - Датская крона / Фунт стерлингов Соединенного королевства со значением 0.8902765649110639



In [ ]:
date_start = "02/03/2002"
date_to = "02/03/2003"
print(calculate_currency_correspondence(currency_codes, date_start, date_to))

Наименее зависимые валюты за период 02/03/2002 - 02/03/2003 - Доллар США / Австралийский доллар со значением 0.7293659242034191
Наиболее зависимые валюты за период 02/03/2002 - 02/03/2003 - Датская крона / Исландская крона со значением 0.984724456505522



In [ ]:
date_start = "02/03/2003"
date_to = "02/03/2004"
print(calculate_currency_correspondence(currency_codes, date_start, date_to))

Наименее зависимые валюты за период 02/03/2003 - 02/03/2004 - Доллар США / Австралийский доллар со значением -0.9232001528010093
Наиболее зависимые валюты за период 02/03/2003 - 02/03/2004 - Датская крона / Фунт стерлингов Соединенного королевства со значением 0.8359446531680941

